## Notebook to test SfM pipeline

1. The repo is already a git submodule so just git submodule update --init --recursive. 
2. Follow the other steps to get the dependencies here: https://www.opensfm.org/docs/building.html, use this file for installing dependencies https://github.com/paulinus/opensfm-docker-base/blob/master/Dockerfile.python3.
    1. Heads up you'll want to do the pip install from within the lcnn conda environment (or another one).
3. To test your install run:
    1. bin/opensfm_run_all data/berlin from the open_sfm directory
    2. python -m http.server and hit the link here: http://localhost:8000/viewer/reconstruction.html#file=/data/berlin/reconstruction.meshed.json

In [22]:
import sys, cv2

To test your own images 

1. run 'python video_converter.py --filename {filename} --n {save_every_n_images} --path {path_to_video}'
2. copy the config.yaml from OpenSfM/data/berlin to your data directory

In [23]:
%%bash 
../OpenSfM/bin/opensfm extract_metadata ../data/lounge

2020-04-15 17:35:36,646 INFO: Loading existing EXIF for img_0.png
2020-04-15 17:35:36,646 INFO: Loading existing EXIF for img_2.png
2020-04-15 17:35:36,646 INFO: Loading existing EXIF for img_5.png
2020-04-15 17:35:36,646 INFO: Loading existing EXIF for img_4.png
2020-04-15 17:35:36,646 INFO: Loading existing EXIF for img_3.png
2020-04-15 17:35:36,646 INFO: Loading existing EXIF for img_7.png
2020-04-15 17:35:36,646 INFO: Loading existing EXIF for img_8.png
2020-04-15 17:35:36,646 INFO: Loading existing EXIF for img_1.png
2020-04-15 17:35:36,647 INFO: Loading existing EXIF for img_6.png


In [24]:
%%bash 
../OpenSfM/bin/opensfm detect_features ../data/lounge

2020-04-15 17:35:39,506 INFO: Skip recomputing ROOT_HAHOG features for image img_4.png
2020-04-15 17:35:39,506 INFO: Skip recomputing ROOT_HAHOG features for image img_0.png
2020-04-15 17:35:39,528 INFO: Skip recomputing ROOT_HAHOG features for image img_6.png
2020-04-15 17:35:39,538 INFO: Skip recomputing ROOT_HAHOG features for image img_3.png
2020-04-15 17:35:39,561 INFO: Skip recomputing ROOT_HAHOG features for image img_7.png
2020-04-15 17:35:39,569 INFO: Skip recomputing ROOT_HAHOG features for image img_5.png
2020-04-15 17:35:39,587 INFO: Skip recomputing ROOT_HAHOG features for image img_2.png
2020-04-15 17:35:39,617 INFO: Skip recomputing ROOT_HAHOG features for image img_8.png
2020-04-15 17:35:39,621 INFO: Skip recomputing ROOT_HAHOG features for image img_1.png
